In [3]:
from google.colab import drive
drive.mount('/content/drive')

path = "drive/My Drive/FiQA/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from evaluate import *

In [4]:
import pickle
from collections import Counter
from tqdm import tqdm
import itertools
import pandas as pd
from itertools import islice
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def remove_empty(test_set):
    for index, row in enumerate(test_set):
        for doc in row[1]:
            if doc in empty_docs:
                del test_set[index]
    return test_set

def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

def save_pickle(path, data):
    with open(path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

Using TensorFlow backend.


In [0]:
# dict mapping of token to idx
vocab = load_pickle(path + 'vocab_full.pickle')
# dict mapping of docid to doc text
docid_to_text = load_pickle(path + 'label_ans.pickle')

# dict mapping of qid to question text
qid_to_text = load_pickle(path + 'qid_text.pickle')

train_qid_rel = load_pickle(path + "qid_rel_train.pickle")
test_qid_rel = load_pickle(path + "qid_rel_test.pickle")
valid_qid_rel = load_pickle(path + "qid_rel_valid.pickle")

train_set = load_pickle(path + 'data/data_train_50.pickle')
valid_set = load_pickle(path + 'data/data_valid_50.pickle')

# train_set = load_pickle(path + 'data/data_train_100.pickle')
# valid_set = load_pickle(path + 'data/data_valid_100.pickle')

# train_set = load_pickle(path + 'data/data_train_200.pickle')
# valid_set = load_pickle(path + 'data/data_valid_200.pickle')

test_set = load_pickle(path + 'data/data_test_500_rel.pickle')
test_set_full = load_pickle(path + 'data/data_test_500.pickle')

empty_docs = load_pickle(path+'empty_docs.pickle')

In [208]:
train_set = [x for x in train_set if x[1] not in empty_docs]
valid_set = [x for x in valid_set if x[1] not in empty_docs]

test_set = remove_empty(test_set)
test_set_full = remove_empty(test_set_full)

print("Number of training samples: {}".format(len(train_set)))
print("Number of validation samples: {}".format(len(valid_set)))
print("Number of test samples: {}".format(len(test_set)))

Number of training samples: 283707
Number of validation samples: 31582
Number of test samples: 330


In [0]:
# train_set = load_pickle(path + 'data-bert/data_train_50.pickle')
# empty_docs = load_pickle(path+'empty_docs.pickle')
# train_set = [x for x in train_set if x[1] not in empty_docs]

In [8]:
!pip install transformers

import torch
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [0]:
# collection = pd.read_csv(path+"data-bert/collection_new.tsv", sep="\t", header=None)
# collection = collection.rename(columns={0: 'docid', 1: 'doc'})

# def load_questions(path):
#     """
#     Returns a dataframe of cols: qid, question
#     """
#     # Question ID and Question text
#     query_df = pd.read_csv(path, sep="\t")
#     queries = query_df[['qid', 'question']]

#     return queries

# queries = load_questions(path + "FiQA_train_question_final.tsv")

# queries['tokenized_q'] = queries['question'].apply(lambda x: tokenizer.tokenize(x))
# queries['len'] = queries['tokenized_q'].apply(lambda x: len(x))

# collection['tokenized_a'] = collection['doc'].apply(lambda x: tokenizer.tokenize(x))
# collection['len'] = collection['tokenized_a'].apply(lambda x: len(x))

# label_to_ans = {}

# for index, row in collection.iterrows():
#     label_to_ans[row['docid']] = row['tokenized_a']

# # Question to question text
# qid_to_text = {}

# for index, row in queries.iterrows():
#     qid_to_text[row['qid']] = row['tokenized_q']

# # save_pickle(path+"data-bert/label_to_ans.pickle", label_to_ans)
# # save_pickle(path+"data-bert/qid_to_text.pickle", qid_to_text)

In [0]:
# print("Max answer len: {}".format(collection['len'].max()))
# print("Min answer len: {}".format(collection['len'].min()))
# print("Average answer len: {}".format(collection['len'].mean()))

# print("Max question len: {}".format(queries['len'].max()))
# print("Min question len: {}".format(queries['len'].min()))
# print("Average question len: {}".format(queries['len'].mean()))

In [0]:
label_to_ans = load_pickle(path+"data-bert/label_to_ans.pickle")
qid_to_text = load_pickle(path+"data-bert/qid_to_text.pickle")

In [0]:
def add_question_token(q_tokens):
    c = ["[CLS]"]
    s = ["[SEP]"]
    q_tokens = c + q_tokens
    q_tokens = q_tokens + s

    return q_tokens

def add_ans_token(a_tokens):
    s = ["[SEP]"]
    a_tokens = a_tokens + s

    return a_tokens

def clip(lst):
    max_seq_len = 256
    if len(lst) > max_seq_len:
        lst = lst[:max_seq_len]
    else:
        lst = lst
    
    return lst

def get_input_ids(sequences, max_seq_len):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []

    for seq in sequences:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Map tokens to their IDs.
        encoded_seq = tokenizer.convert_tokens_to_ids(seq)
        
        # Add the encoded sentence to the list.
        input_ids.append(encoded_seq)

    input_ids = pad_sequences(input_ids, maxlen=max_seq_len, dtype="long", 
                          value=0, truncating="post", padding="post")
    return input_ids

def get_att_mask(input_ids):
    # Create attention masks
    attention_masks = []

    # For each sentence...
    for sent in input_ids:
        
        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]
        
        # Store the attention mask for this sentence.
        attention_masks.append(att_mask)

    return attention_masks

In [0]:
def get_sequence_df(dataset):
    df = pd.DataFrame(dataset)
    df = df.rename(columns={0: 'qid', 1: 'pos', 2:'neg'})
    df_pos = df[['qid', 'pos']]
    df_pos = df_pos.rename(columns={'pos': 'docid'})
    df_pos['label'] = df_pos.apply(lambda x: 1, axis=1)
    df_pos = df_pos.drop_duplicates()

    df_neg = df[['qid', 'neg']]
    df_neg = df_neg.rename(columns={'neg': 'docid'})
    df_neg['label'] = df_neg.apply(lambda x: 0, axis=1)
    data_df = pd.concat([df_pos, df_neg]).sort_values(by=['qid'])

    data_df['question'] = data_df['qid'].apply(lambda x: qid_to_text[x])
    data_df['ans_cand'] = data_df['docid'].apply(lambda x: label_to_ans[x])
    data_df['ques_token'] = data_df['question'].apply(lambda x: add_question_token(x))
    data_df['ans_cand'] = data_df['ans_cand'].apply(lambda x: add_ans_token(x))

    data_df = data_df[['qid', 'docid', 'label', 'ans_cand','ques_token']]
    data_df['seq'] = data_df['ques_token'] + data_df['ans_cand']

    max_seq_len = 256

    data_df['seq_clipped'] = data_df['seq'].apply(clip)
    # train['len'] = train['seq_clipped'].apply(lambda x: len(x))

    return data_df

In [165]:
train = get_sequence_df(train_set)
valid = get_sequence_df(valid_set)

# Get the lists of sentences and their labels.
train_sequences = train.seq_clipped.values
train_labels = train.label.values

valid_sequences = valid.seq_clipped.values
valid_labels = valid.label.values

print(len(train_sequences))
print(len(valid_sequences))

train_sequences = train_sequences[:500]
train_labels = train_labels[:500]

valid_sequences = valid_sequences[:500]
valid_labels = valid_labels[:500]

max_seq_len = 256

train_input = get_input_ids(train_sequences, max_seq_len)
valid_input = get_input_ids(valid_sequences, max_seq_len)

print(train_input[0])

train_att_mask = get_att_mask(train_input)
valid_att_mask = get_att_mask(valid_input)

# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_input)
validation_inputs = torch.tensor(valid_input)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(valid_labels)

train_masks = torch.tensor(train_att_mask)
validation_masks = torch.tensor(valid_att_mask)

298401
33143
[  101  2054  2003  2641  1037  2449 10961  2006  1037  2449  4440  1029
   102  1996 25760  8606 20246  2000  1996  3395  1012  1999  2236  1996
  2190  1045  2064  2360  2003  2115  2449 10961  2089  2022  2139  8566
  6593  7028  1012  2021  2009  9041  2006  1996  6214  1998  2054  2009
  2003  2017  2215  2000  2139  8566  6593  1012  3604 26457  2040  3604
  2185  2013  2188  2006  2449  2089  2139  8566  6593  3141 11727  1010
  2164  1996  3465  1997  4285  2037  7688  1010  1996  3465  1997 26859
  1998 12278  1998  2060  6623  1998  4072 11727  1012 26457  2024  2641
  1523  7118  2185  2013  2188  1524  2065  2037  5704  5478  2068  2000
  2022  2185  2013  2188 12381  2936  2084  2019  6623  2154  1521  1055
  2147  1998  2027  2342  2000  3637  2030  2717  2000  3113  1996  7670
  1997  2037  2147  1012  1996  5025  3465  1997 12278  1998  5043  2389
 11727  2089  2022  2139 29510  2030  1996 26980  2089  2224  1037  3115
  7954 21447  1998  4359  2501  4363  

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [16]:
print(len(train_inputs))
print(len(valid_input))

50000
5000


In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [19]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [22]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [23]:
len(train_dataloader)

1563

In [0]:

# import random

# # This training code is based on the `run_glue.py` script here:
# # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# # Set the seed value all over the place to make this reproducible.
# seed_val = 42

# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)

# # Store the average loss after each epoch so we can plot them.
# loss_values = []

# # For each epoch...
# for epoch_i in range(0, epochs):
    
#     # ========================================
#     #               Training
#     # ========================================
    
#     # Perform one full pass over the training set.

#     print("")
#     print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
#     print('Training...')

#     # Measure how long the training epoch takes.
#     t0 = time.time()

#     # Reset the total loss for this epoch.
#     total_loss = 0

#     # Put the model into training mode. Don't be mislead--the call to 
#     # `train` just changes the *mode*, it doesn't *perform* the training.
#     # `dropout` and `batchnorm` layers behave differently during training
#     # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
#     model.train()

#     # For each batch of training data...
#     for step, batch in enumerate(train_dataloader):

#         # Progress update every 40 batches.
#         if step % 40 == 0 and not step == 0:
#             # Calculate elapsed time in minutes.
#             elapsed = format_time(time.time() - t0)
            
#             # Report progress.
#             print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

#         # Unpack this training batch from our dataloader. 
#         #
#         # As we unpack the batch, we'll also copy each tensor to the GPU using the 
#         # `to` method.
#         #
#         # `batch` contains three pytorch tensors:
#         #   [0]: input ids 
#         #   [1]: attention masks
#         #   [2]: labels 
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)

#         # Always clear any previously calculated gradients before performing a
#         # backward pass. PyTorch doesn't do this automatically because 
#         # accumulating the gradients is "convenient while training RNNs". 
#         # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
#         model.zero_grad()        

#         # Perform a forward pass (evaluate the model on this training batch).
#         # This will return the loss (rather than the model output) because we
#         # have provided the `labels`.
#         # The documentation for this `model` function is here: 
#         # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
#         outputs = model(b_input_ids, 
#                     token_type_ids=None, 
#                     attention_mask=b_input_mask, 
#                     labels=b_labels)
#                 # The call to `model` always returns a tuple, so we need to pull the 
#         # loss value out of the tuple.
#         loss = outputs[0]

#         # Accumulate the training loss over all of the batches so that we can
#         # calculate the average loss at the end. `loss` is a Tensor containing a
#         # single value; the `.item()` function just returns the Python value 
#         # from the tensor.
#         total_loss += loss.item()

#         # Perform a backward pass to calculate the gradients.
#         loss.backward()

#         # Clip the norm of the gradients to 1.0.
#         # This is to help prevent the "exploding gradients" problem.
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#         # Update parameters and take a step using the computed gradient.
#         # The optimizer dictates the "update rule"--how the parameters are
#         # modified based on their gradients, the learning rate, etc.
#         optimizer.step()

#         # Update the learning rate.
#         scheduler.step()

#     # Calculate the average loss over the training data.
#     avg_train_loss = total_loss / len(train_dataloader)            
    
#     # Store the loss value for plotting the learning curve.
#     loss_values.append(avg_train_loss)

#     print("")
#     print("  Average training loss: {0:.2f}".format(avg_train_loss))
#     print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
#     # ========================================
#     #               Validation
#     # ========================================
#     # After the completion of each training epoch, measure our performance on
#     # our validation set.

#     print("")
#     print("Running Validation...")

#     t0 = time.time()

#     # Put the model in evaluation mode--the dropout layers behave differently
#     # during evaluation.
#     model.eval()

#     # Tracking variables 
#     eval_loss, eval_accuracy = 0, 0
#     nb_eval_steps, nb_eval_examples = 0, 0

#     # Evaluate data for one epoch
#     for batch in validation_dataloader:
        
#         # Add batch to GPU
#         batch = tuple(t.to(device) for t in batch)
        
#         # Unpack the inputs from our dataloader
#         b_input_ids, b_input_mask, b_labels = batch
        
#         # Telling the model not to compute or store gradients, saving memory and
#         # speeding up validation
#         with torch.no_grad():        

#             # Forward pass, calculate logit predictions.
#             # This will return the logits rather than the loss because we have
#             # not provided labels.
#             # token_type_ids is the same as the "segment ids", which 
#             # differentiates sentence 1 and 2 in 2-sentence tasks.
#             # The documentation for this `model` function is here: 
#             # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
#             outputs = model(b_input_ids, 
#                             token_type_ids=None, 
#                             attention_mask=b_input_mask)
        
#         # Get the "logits" output by the model. The "logits" are the output
#         # values prior to applying an activation function like the softmax.
#         logits = outputs[0]

#         # Move logits and labels to CPU
#         logits = logits.detach().cpu().numpy()
#         label_ids = b_labels.to('cpu').numpy()
        
#         # Calculate the accuracy for this batch of test sentences.
#         tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
#         # Accumulate the total accuracy.
#         eval_accuracy += tmp_eval_accuracy

#         # Track the number of batches
#         nb_eval_steps += 1

#     # Report the final accuracy for this validation run.
#     print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
#     print("  Validation took: {:}".format(format_time(time.time() - t0)))

# print("")
# print("Training complete!")


In [0]:
# torch.save(model.state_dict(), 'bert-model.pt')

In [25]:
model.load_state_dict(torch.load(path + 'model/bert-model.pt'))

<All keys matched successfully>

In [0]:
for row in test_set:
    row[2] = [x for x in row[2] if x is not 0]

In [210]:
print(test_set[0])

[1, [14255], [84963, 354716, 522619, 418999, 322064, 141738, 303078, 355897, 71987, 219313, 310612, 257168, 410431, 541809, 466718, 89190, 66356, 329209, 283505, 596289, 362060, 113632, 113776, 362069, 129355, 381151, 81343, 46791, 360925, 294738, 157233, 263521, 509862, 90290, 598547, 397608, 442968, 81599, 68969, 434619, 71569, 476980, 365558, 251392, 118615, 292748, 133701, 246461, 327002, 510692, 368263, 304452, 224167, 234436, 98636, 107794, 388713, 524879, 399762, 174321, 282958, 245447, 345070, 361978, 183612, 566417, 539511, 19640, 420295, 525149, 302049, 12729, 223170, 41793, 356884, 288995, 472824, 597053, 40628, 546277, 262960, 275312, 112793, 248448, 363495, 223697, 406789, 349674, 550345, 11132, 265527, 189642, 18934, 527776, 314161, 576985, 146388, 156554, 424720, 109546, 531442, 144190, 399882, 469043, 89008, 494000, 296345, 193367, 299971, 234743, 189765, 354314, 128861, 431010, 158409, 523564, 177074, 257738, 213041, 283113, 187227, 533808, 444899, 507107, 38249, 20217

In [0]:
test_set_small = test_set

In [0]:
for row in test_set:
    row[2] = [x for x in row[2] if x not in row[1]]

In [68]:
p = 0
n = 0

for row in test_set:
    p += len(row[1])
    n += len(row[2])

print(p)
print(n)

814
164107


In [75]:
test_df = pd.DataFrame(test_set)
test_df = test_df.rename(columns={0: 'qid', 1: 'pos', 2:'cand'})
# test_df = test_df[['qid', 'cand']]

test_df.head(5)

,qid,pos,cand
0,1,[14255],"[84963, 354716, 522619, 418999, 322064, 141738..."
1,25,"[107584, 562777]","[426796, 350819, 45373, 531137, 407547, 443960..."
2,43,[76662],"[226542, 581041, 483305, 311971, 30935, 72854,..."
3,80,[252473],"[158646, 559539, 236961, 519473, 540688, 11118..."
4,460,"[591357, 556021, 555097]","[77564, 111240, 104043, 141579, 591163, 510989..."


In [76]:
test_pos = test_df[['qid', 'pos']]
test_pos = test_pos.explode('pos')
test_pos = test_pos.rename(columns={'pos': 'docid'})
test_pos['label'] = test_pos.apply(lambda x: 1, axis=1)

len(test_pos)

814

In [80]:
test_neg = test_df[['qid', 'cand']]
test_neg = test_neg.explode('cand')
test_neg = test_neg.rename(columns={'cand': 'docid'})
test_neg ['label'] = test_neg .apply(lambda x: 0, axis=1)

test_neg.head(5)

,qid,docid,label
0,1,84963,0
0,1,354716,0
0,1,522619,0
0,1,418999,0
0,1,322064,0


In [81]:
len(test_neg)

164107

In [82]:
test_data = pd.concat([test_pos, test_neg]).sort_values(by=['qid'])

test_data['question'] = test_data['qid'].apply(lambda x: qid_to_text[x])
test_data['ans_cand'] = test_data['docid'].apply(lambda x: label_to_ans[x])
test_data['ques_token'] = test_data['question'].apply(lambda x: add_question_token(x))
test_data['ans_cand'] = test_data['ans_cand'].apply(lambda x: add_ans_token(x))

test_data = test_data[['qid', 'docid', 'label', 'ans_cand','ques_token']]
test_data['seq'] = test_data['ques_token'] + test_data['ans_cand']
test_data['seq_clipped'] = test_data['seq'].apply(clip)

test_data.head(5)

,qid,docid,label,ans_cand,ques_token,seq,seq_clipped
0,1,14255,1,"[yes, you, can, claim, your, business, de, ##d...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ..."
0,1,581265,0,"[in, the, us, tax, system, ,, you, cannot, "", ...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ..."
0,1,515690,0,"[i, don, ', t, quite, understand, your, though...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ..."
0,1,402437,0,"[&, gt, ;, the, base, value, from, infrastruct...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ..."
0,1,373481,0,"[the, short, answer, is, yes, ,, losses, get, ...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ...","[[CLS], claiming, business, expenses, for, a, ..."


In [0]:
docid_map = test_data[['docid', 'seq_clipped']]
test_docid_to_seq = {}

for index, row in docid_map.iterrows():
    test_docid_to_seq[row['docid']] = row['seq_clipped']


In [148]:
print(take(5, test_docid_to_seq.items()))

save_pickle(path+'data-bert/test_docid_to_seq.pickle', test_docid_to_seq)

[(14255, ['[CLS]', 'what', 'is', 'the', 'easiest', 'way', 'to', 'back', '-', 'test', 'index', 'funds', 'and', 'et', '##fs', '?', '[SEP]', 'yes', 'you', 'can', 'claim', 'your', 'business', 'de', '##duction', '##s', 'if', 'you', 'are', 'not', 'making', 'any', 'income', 'yet', '.', 'but', 'first', 'you', 'should', 'decide', 'what', 'structure', 'you', 'want', 'to', 'have', 'for', 'your', 'business', '.', 'either', 'a', 'company', 'structure', 'or', 'a', 'sole', 'trader', 'or', 'partnership', '.', 'company', 'structure', 'if', 'you', 'choose', 'a', 'company', 'structure', '(', 'which', 'is', 'more', 'expensive', 'to', 'set', 'up', ')', 'you', 'would', 'claim', 'your', 'de', '##duction', '##s', 'but', 'no', 'income', '.', 'so', 'you', 'would', 'be', 'making', 'a', 'loss', ',', 'and', 'continue', 'making', 'losses', 'until', 'your', 'income', 'from', 'the', 'business', 'exceed', 'your', 'expenses', '.', 'so', 'these', 'losses', 'will', 'remain', 'inside', 'the', 'company', 'and', 'can', 'be'

In [0]:
test_d = test_data[['qid', 'docid', 'label']]
testset = []

for index, row in test_d.iterrows():
    tmp = []
    tmp.append(row['qid'])
    tmp.append(row['docid'])
    tmp.append(row['label'])

    testset.append(tmp)

In [0]:
t = test_set_small[:1]

In [176]:
text = test_docid_to_seq[14255]
 
encoded_seq = tokenizer.convert_tokens_to_ids(text)

print(encoded_seq)

def pad_seq(seq, max_seq_len):
    # Pad each seq to be the same length to process in batch.
    # pad_token = 0
    if len(seq) >= max_seq_len:
        seq = seq[:max_seq_len]
    else:
        seq += [0]*(max_seq_len - len(seq))
    return seq

padded_seq_idx = pad_seq(encoded_seq, 500)

print([int(token_id > 0) for token_id in padded_seq_idx])

[101, 2054, 2003, 1996, 25551, 2126, 2000, 2067, 1011, 3231, 5950, 5029, 1998, 3802, 10343, 1029, 102, 2748, 2017, 2064, 4366, 2115, 2449, 2139, 16256, 2015, 2065, 2017, 2024, 2025, 2437, 2151, 3318, 2664, 1012, 2021, 2034, 2017, 2323, 5630, 2054, 3252, 2017, 2215, 2000, 2031, 2005, 2115, 2449, 1012, 2593, 1037, 2194, 3252, 2030, 1037, 7082, 17667, 2030, 5386, 1012, 2194, 3252, 2065, 2017, 5454, 1037, 2194, 3252, 1006, 2029, 2003, 2062, 6450, 2000, 2275, 2039, 1007, 2017, 2052, 4366, 2115, 2139, 16256, 2015, 2021, 2053, 3318, 1012, 2061, 2017, 2052, 2022, 2437, 1037, 3279, 1010, 1998, 3613, 2437, 6409, 2127, 2115, 3318, 2013, 1996, 2449, 13467, 2115, 11727, 1012, 2061, 2122, 6409, 2097, 3961, 2503, 1996, 2194, 1998, 2064, 2022, 3344, 2830, 2000, 2925, 3318, 2086, 2043, 2017, 2024, 2437, 11372, 2000, 16396, 2122, 11372, 1012, 6523, 2000, 2012, 2080, 1011, 2194, 4171, 6409, 2005, 2062, 2592, 1012, 7082, 17667, 1997, 5386, 3252, 2065, 2017, 5454, 2000, 2022, 1037, 7082, 17667, 2030, 1037,

In [0]:
def eval(model, test_set, qid_rel, max_seq_len, k):
    qid_pred_rank = {}

    model.eval()

    for i, seq in enumerate(tqdm(test_set)):
        
        qid, label, cands = seq[0], seq[1], seq[2]

        cands_id = np.array(cands)

        scores = []

        for docid in cands:

            seq_text = test_docid_to_seq[docid]

            encoded_seq = tokenizer.convert_tokens_to_ids(seq_text)

            input_ids = pad_seq(encoded_seq, max_seq_len)

            att_mask = torch.tensor([[int(token_id > 0) for token_id in input_ids]]).to(device)
            
            input_ids = torch.tensor([input_ids]).to(device)


            with torch.no_grad():
            # Forward pass, calculate logit predictions
                outputs = model(input_ids, token_type_ids=None, attention_mask=att_mask)

            logits = outputs[0]

            pred = torch.sigmoid(logits)

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            pred = pred.detach().cpu().numpy()

            scores.append(pred[:,1][0])

        # Get the indices of the sorted similarity scores
        sorted_index = np.argsort(scores)[::-1]

        # Get the docid from the sorted indices
        ranked_ans = cands_id[sorted_index]

        # Dict - key: qid, value: ranked list of docids
        qid_pred_rank[qid] = ranked_ans

        # return qid_pred_rank
        MRR, average_ndcg, precision = evaluate(qid_pred_rank, qid_rel, k)

        return qid_pred_rank, MRR, average_ndcg, precision

In [238]:
toy_test_label = dict(itertools.islice(test_qid_rel.items(), 1))

{1: [14255]}

In [243]:
model.load_state_dict(torch.load(path + 'model/bert-model.pt'))

k = 10

qid_pred_rank, MRR, average_ndcg, precision = eval(model, t, toy_test_label, 256, k)

num_q = len(test_set)

print("\n\nAverage nDCG@{} for {} queries: {}\n".format(k, num_q, average_ndcg))

print("MRR@{} for {} queries: {}\n".format(k, num_q, MRR))

print("Average Precision@{}: {}".format(1, precision))

  0%|          | 0/1 [00:00<?, ?it/s]



Average nDCG@10 for 330 queries: 0

MRR@10 for 330 queries: 0.0

Average Precision@1: 0.0
